# Gas Turbine

In this section we will be investigating gas turbine cycles using TESPy. In the first step the ideal Brayton cycle is
modeled. The ideal Brayton cycle consists of three subprocesses, i.e. isentropic compression, isobaric heat transfer
and isentropic expansion (see {numref}`brayton-flowsheet`). Next, we are comparing the cycle to the open cycle gas 
turbine, which instead of isobaric heat transfer uses a combustion chamber.

## Ideal Brayton Cycle

The flowsheet of the Brayton cycle is shown in {numref}`brayton-flowsheet`. Based on the flowsheet we can calculate the
thermal efficiency of the cycle using eq. {eq}`brayton-thermal-efficiency`. For the exergy analysis we have to define
the plant's fuel and product exergy as well as the exergy losses in eqs. {eq}`brayton-fuel-exergy`,
{eq}`brayton-product-exergy` and {eq}`brayton-exergy-loss`.

```{math}
:label: brayton-thermal-efficiency
\eta_\text{th} = \frac{|\sum \dot W|}{\dot Q_\text{heater}}
```

```{math}
:label: brayton-fuel-exergy
\dot E_\text{F,tot} = \dot E_\text{F,heater}
```

```{math}
:label: brayton-product-exergy
\dot E_\text{P,tot} = \dot E_\text{P,turbine} - \dot E_\text{F,compressor}
```
 
```{math}
:label: brayton-exergy-loss
\dot E_\text{L,tot} = \dot m \cdot e_\text{4}
```

```{figure} /figures/brayton.svg
---
name: brayton-flowsheet
---
Flow sheet of the Brayton cycle.
```

{numref}`brayton-cycle-param` indicates the parameter settings for the Brayton cycle for the initial setup.

```{list-table} Parametrization of the Brayton cycle
:header-rows: 1
:name: brayton-cycle-param

* - Location
  - Parameter
  - Value
  - Unit
* - Compressor
  - Isentropic efficiency
  - 100
  - \%
* - 
  - Pressure ratio
  - 10
  - 
* - Heater
  - Pressure ratio
  - 1
  - 
* - Turbine
  - Isentropic efficiency
  - 100
  - \%
* - Generator
  - Total power output
  - -100
  - MW
* - 1
  - Pressure
  - 1.013
  - bar
* - 
  - Temperature
  - 20
  - °C
* - 3
  - Temperature
  - 1400
  - °C
* - 4
  - Pressure
  - 1.013
  - bar
```

## Excercise 1

1. Build the TESPy model according to the flowsheet of the ideal Brayton cycle and its specifications shown in
   {numref}`brayton-cycle-param`:
   - Calculate the thermal efficiency of the process.
   - Add the setup for the exergy analysis and run it:
     - What is the total exergy efficiency?
     - What is the total exergy destruction?
     - What are the exergy losses to the ambient?
   - Why is it impossible to reach an exergy efficiency of 100 %?
2. Make individual parameter variations for the compressor pressure ratio and the turbine inlet temperature:
   - Calculate thermal efficiency and exergy efficiency for every combination of values.
   - Make a 3D-plot showing the thermal efficiency
   - Make a 3D-plot showing the exergy efficiency

### Solution 1

We can set up the plant with standard components. The heater of the Brayton cycle can be represented by the
`HeatExchangerSimple` component. 

In [ ]:
from tespy.components import Source, Sink, Compressor, Turbine, HeatExchangerSimple
from tespy.connections import Connection, Bus
from tespy.networks import Network
from tespy.tools import ExergyAnalysis


nwk = Network(fluids=["Air"], p_unit="bar", T_unit="C", iterinfo=False)

p0 = 1.013
T0 = 20

so = Source("air intake")
cp = Compressor("compressor")
hi = HeatExchangerSimple("heater")
tu = Turbine("turbine")
si = Sink("air outlet")

c1 = Connection(so, "out1", cp, "in1", label="1")
c2 = Connection(cp, "out1", hi, "in1", label="2")
c3 = Connection(hi, "out1", tu, "in1", label="3")
c4 = Connection(tu, "out1", si, "in1", label="4")

nwk.add_conns(c1, c2, c3, c4)

power_output = Bus("power output")
power_output.add_comps(
    {"comp": cp, "base": "bus"},
    {"comp": tu, "base": "component"}
)

heat_input = Bus("heat input")
heat_input.add_comps(
    {"comp": hi, "base": "bus"},
)

chimney = Bus("chimney")
chimney.add_comps(
    {"comp": si, "base": "component"}
)
nwk.add_busses(power_output, heat_input, chimney)

c1.set_attr(fluid={"Air": 1}, p=p0, T=T0)
c3.set_attr(T=1400)
c4.set_attr(p=p0)

cp.set_attr(eta_s=1, pr=10)
hi.set_attr(pr=1)
tu.set_attr(eta_s=1)

power_output.set_attr(P=-100e6)

nwk.solve("design")

thermal_efficiency = abs(power_output.P.val) / heat_input.P.val

ean = ExergyAnalysis(nwk, E_F=[heat_input], E_P=[power_output], E_L=[chimney])

ean.analyse(p0, T0)

In [ ]:
from myst_nb import glue
glue("brayton-thermal-efficiency", thermal_efficiency * 100, display=False)
glue("brayton-exergy-efficiency", ean.network_data.loc["epsilon"] * 100, display=False)

The cycle's efficiency is at about {glue:text}`brayton-thermal-efficiency:.2f` %, with an exergy efficiency of 
{glue:text}`brayton-exergy-efficiency:.2f` %. We can extract the plant's overall exergy analysis results from the 
`network_data` attribute:

In [ ]:
ean.network_data.to_frame().transpose().rename({0: "value"})

We can see the 



### Solution 2

### Solution 3

## Open Cycle Gas Turbine

```{figure} /figures/gas_turbine.svg
---
name: gas-turbine-flowsheet
---
Flow sheet of the open cycle gas turbine.
```

- BK mit Verbrennung -> Chemical exergy
- Wie funktioniert die chemische Exergie?

Table of parameters for gas turbine 

## Excercise 2

4. Build the TESPy model for the open cycle gas turbine and specify the process and component parameters identically to
   the setup of the Brayton cycle.
   - Make a comparison of the results:
     - Air mass flow
     - Compressor outlet temperature
     - Turbine outlet temperature
     - Total heat input
5. Run the exergy analysis for the open cycle gas turbine:
   - Why is the exergy 
6. Make a parametric analysis of
   - the compressor isentropic efficiency,
   - the turbine isentropic efficiency.
   Why is the effect of the compressor efficiency stronger than the effect of the turbine efficieny?

7. Make a parametric analysis of
   - pressure losses in the combustion chamber,
   - heat losses in the combustion chamber.